In [ ]:
import pandas as pd 
import pyodbc

Load the data to inspect

In [2]:
netflix_data = pd.read_csv("netflix_titles.csv")

In [3]:
netflix_data.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."


In [4]:
netflix_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


Transformation step: handle missing values, remove duplicates, and standardize formats

In [5]:
# Fill missing values for specific columns with a placeholder
transformed_data = netflix_data.fillna({
    'director': 'Johnson',
    'cast': 'Ranveer',
    'country': 'India',
    'date_added': 'Unknown',
    'rating': 'Not Rated',
    'duration': '60 min'
})

In [6]:
# Remove duplicates based on the 'show_id' column
transformed_data = transformed_data.drop_duplicates(subset=['show_id'])

In [7]:
# Convert 'date_added' to a standardized datetime format if not 'Unknown'
transformed_data['date_added'] = transformed_data['date_added'].apply(
    lambda x: pd.to_datetime(x, errors='coerce') if x != 'Unknown' else None
)
transformed_data = transformed_data.dropna(subset=['date_added'])

In [8]:
# Inspect the transformed data
transformed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8797 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       8797 non-null   object        
 1   type          8797 non-null   object        
 2   title         8797 non-null   object        
 3   director      8797 non-null   object        
 4   cast          8797 non-null   object        
 5   country       8797 non-null   object        
 6   date_added    8797 non-null   datetime64[ns]
 7   release_year  8797 non-null   int64         
 8   rating        8797 non-null   object        
 9   duration      8797 non-null   object        
 10  listed_in     8797 non-null   object        
 11  description   8797 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 893.4+ KB


Load to SQL Server

In [9]:
# Step 1: Create the SQL Server table
def create_table_sql_server(server_name, database_name):
    try:
        # Establish a connection using Windows Authentication
        conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes;")
        cursor = conn.cursor()
        print(cursor)
        
        # Create table
        create_table_query = """
        CREATE TABLE netflix_titles (
            show_id NVARCHAR(50) PRIMARY KEY,
            type NVARCHAR(50),
            title NVARCHAR(MAX),
            director NVARCHAR(MAX),
            cast NVARCHAR(MAX),
            country NVARCHAR(MAX),
            date_added DATE,
            release_year INT,
            rating NVARCHAR(50),
            duration NVARCHAR(50),
            listed_in NVARCHAR(MAX),
            description NVARCHAR(MAX)
        )
        """
        cursor.execute(create_table_query)
        conn.commit()
        print("Table created successfully in SQL Server!")
    except Exception as e:
        print(f"Error creating table: {e}")
    finally:
        cursor.close()
        conn.close()


In [ ]:
# Step 2: Load data into SQL Server
def load_data_sql_server(server_name, database_name, data):
    try:
        # Establish a connection using Windows Authentication
        conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes;")
        cursor = conn.cursor()
        
        # Insert data into the table
        insert_query = """
        INSERT INTO netflix_titles (
            show_id, type, title, director, cast, country, date_added,
            release_year, rating, duration, listed_in, description
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        for _, row in data.iterrows():
            cursor.execute(insert_query, (
                row['show_id'], row['type'], row['title'], row['director'],
                row['cast'], row['country'], row['date_added'], row['release_year'],
                row['rating'], row['duration'], row['listed_in'], row['description']
            ))
        
        conn.commit()
        print("Data loaded successfully into SQL Server!")
    except Exception as e:
        print(f"Error loading data: {e}")
    finally:
        cursor.close()
        conn.close()

server_name = 'LAPTOP-Instanceid\SQLEXPRESS'  # Replace with your SQL Server instance name
database_name = 'etl_demo'

create_table_sql_server(server_name, database_name)
load_data_sql_server(server_name, database_name, transformed_data)



Data loaded successfully into SQL Server!
